In [1]:
import json
import os.path
import pandas as pd
import numpy as np
from preprocessing.logParser import openLogs

In [2]:
BASE_DIR = os.path.abspath('test_statistics.ipynb')
BASE_DIR = os.path.dirname(BASE_DIR)
BASE_DIR = BASE_DIR + '/spbu_ACADRU_spring_2018-TL'
cursor = openLogs(BASE_DIR)

In [3]:
#and questions NOT LIKE '__' means that we needn't questions like {}
#and attempts < 4 means that we use only three first attempts, other attempts are not interesting
request = """
        select 
        user_id, attempts, questions,
        page, grade, max_grade
        from problem_check
        where course_name = "course-v1:spbu+ACADRU+spring_2018"
        and questions NOT LIKE '__'
        and attempts < 4
        """.format("https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/b434df438c2045eba63afc089d64b3de/244a81ebf8a64b609cd8587bbe18edc5/")
cursor.execute(request)
data = cursor.fetchall()

columns_names = []
for i in cursor.description[:]:
    columns_names.append(i[0])

df = pd.DataFrame(data=data, columns=columns_names)

### модуль тестов, пункт 1 a: (средний результат за тест и медиана)

In [4]:
listForAverage=df.sort_values(by=['page','attempts'])

In [5]:
average=pd.DataFrame(columns=('page','attempt','median','average_grade','max_grade','numOfSolutions'))

In [6]:
arrayForMedian=[]

In [7]:
prevTest=''
for row in listForAverage.itertuples():
    if not prevTest:
        prevTest=row.page
        prevAttempt=row.attempts
        prevMaxGrade=row.max_grade
        numOfSolutions=0

    if prevTest==row.page:
        if prevAttempt==row.attempts:            
            arrayForMedian.append(row.grade)
            numOfSolutions=numOfSolutions+1
        else:
            arrayForMedian=np.asarray(arrayForMedian)
            mean=np.mean(arrayForMedian)
            median=np.median(arrayForMedian)
            average=average.append({'page':row.page,'attempt':prevAttempt,'median':median,'average_grade':mean,'max_grade':prevMaxGrade,'numOfSolutions':numOfSolutions},ignore_index=True)            
            numOfSolutions=1
            prevAttempt=row.attempts
            prevMaxGrade=row.max_grade
            arrayForMedian=[]
            arrayForMedian.append(row.grade)
    else:  
        arrayForMedian=np.asarray(arrayForMedian)
        mean=np.mean(arrayForMedian)
        median=np.median(arrayForMedian)
        average=average.append({'page':row.page,'attempt':prevAttempt,'median':median,'average_grade':mean,'max_grade':prevMaxGrade,'numOfSolutions':numOfSolutions},ignore_index=True)            
        numOfSolutions=1
        prevAttempt=row.attempts
        prevTest=row.page     
        prevMaxGrade=row.max_grade
        arrayForMedian=[]
        arrayForMedian.append(row.grade)

In [8]:
print(average)

                                                  page attempt  median  \
0    block-v1:spbu+ACADRU+spring_2018+type@problem+...       1     1.0   
1    block-v1:spbu+ACADRU+spring_2018+type@problem+...       1     1.0   
2    block-v1:spbu+ACADRU+spring_2018+type@problem+...       1     0.0   
3    block-v1:spbu+ACADRU+spring_2018+type@problem+...       2     0.0   
4    block-v1:spbu+ACADRU+spring_2018+type@problem+...       1    11.0   
5    block-v1:spbu+ACADRU+spring_2018+type@problem+...       2    12.0   
6    block-v1:spbu+ACADRU+spring_2018+type@problem+...       1     8.0   
7    block-v1:spbu+ACADRU+spring_2018+type@problem+...       2     8.0   
8    block-v1:spbu+ACADRU+spring_2018+type@problem+...       1     1.0   
9    block-v1:spbu+ACADRU+spring_2018+type@problem+...       1     8.0   
10   block-v1:spbu+ACADRU+spring_2018+type@problem+...       2     9.0   
11   block-v1:spbu+ACADRU+spring_2018+type@problem+...       3     9.0   
12   block-v1:spbu+ACADRU+spring_2018+

In [9]:
average=average.to_json(orient='records')

In [10]:
print(average)

[{"page":"block-v1:spbu+ACADRU+spring_2018+type@problem+block@08944f89f5a69eb0f3d9","attempt":1,"median":1.0,"average_grade":0.6666666667,"max_grade":1.0,"numOfSolutions":3},{"page":"block-v1:spbu+ACADRU+spring_2018+type@problem+block@0c2e896b3e86420d854609b41ffb7229","attempt":1,"median":1.0,"average_grade":1.0,"max_grade":1.0,"numOfSolutions":2},{"page":"block-v1:spbu+ACADRU+spring_2018+type@problem+block@0c2e896b3e86420d854609b41ffb7229","attempt":1,"median":0.0,"average_grade":0.0,"max_grade":2.0,"numOfSolutions":1},{"page":"block-v1:spbu+ACADRU+spring_2018+type@problem+block@0cb4a601e614492a8c2df8a7d024a012","attempt":2,"median":0.0,"average_grade":0.0,"max_grade":2.0,"numOfSolutions":1},{"page":"block-v1:spbu+ACADRU+spring_2018+type@problem+block@0cb4a601e614492a8c2df8a7d024a012","attempt":1,"median":11.0,"average_grade":9.7391304348,"max_grade":14.0,"numOfSolutions":23},{"page":"block-v1:spbu+ACADRU+spring_2018+type@problem+block@172408b80f6c43a0b8bebc1848b4808d","attempt":2,"me

### модуль тестов, пункт 1 с: (процент правильных ответов на вопрос)

In [11]:
#parsing of questions string in two columns: question and result
listForQuestions=pd.DataFrame(columns=('question','attempt','user_id','result'))

for rowCounter in range(df.shape[0]):
    currentAttempt=df.attempts[rowCounter]
    currentUser=df.user_id[rowCounter]
    row=(df.questions[rowCounter][1:-1]).split(',')
    for cellRow in row:
        tmprow=cellRow.split(':')
        if tmprow[1]=='True':
            listForQuestions=listForQuestions.append({'question':tmprow[0],'attempt':currentAttempt,'user_id':currentUser,'result':1}, ignore_index=True)
        else:
            listForQuestions=listForQuestions.append({'question':tmprow[0],'attempt':currentAttempt,'user_id':currentUser,'result':0}, ignore_index=True)


In [12]:
listForQuestions['question'] = listForQuestions['question'].map(str.strip)
listForQuestions=listForQuestions.sort_values(by=['question','attempt','user_id'])

In [13]:
percentage=pd.DataFrame(columns=('question','attempt','numOfAnswers','percentageOfRightAnswers'))

In [14]:
prevQuestion=''
for row in listForQuestions.itertuples():
    if not prevQuestion:
        prevQuestion=row.question
        prevAttempt=row.attempt
        tmpUsersList=pd.DataFrame(columns=('user_id','result'))   
    
    if prevQuestion==row.question :        
        if prevAttempt==row.attempt :
            if row.attempt==1:
                tmpUsersList=tmpUsersList.append({'user_id':row.user_id,'result':row.result}, ignore_index=True)
            else:
                for user in tmpUsersList:
                    if user==row.user_id:
                        user.result=row.result #updating result for new attempt
        else:
            percentage=percentage.append({'question':prevQuestion,'attempt':prevAttempt,'numOfAnswers':tmpUsersList.shape[0],'percentageOfRightAnswers':100*(((tmpUsersList.result).sum())/tmpUsersList.shape[0])},ignore_index=True)            
            prevAttempt=row.attempt
    else:        
        percentage=percentage.append({'question':prevQuestion,'attempt':prevAttempt,'numOfAnswers':tmpUsersList.shape[0],'percentageOfRightAnswers':100*(((tmpUsersList.result).sum())/tmpUsersList.shape[0])},ignore_index=True)
        tmpUsersList=pd.DataFrame(columns=('user_id','result')) 
        tmpUsersList=tmpUsersList.append({'user_id':row.user_id,'result':row.result}, ignore_index=True)
        prevQuestion=row.question

In [15]:
print(percentage)

                                    question attempt numOfAnswers  \
0                 "062bec6c9167fb11f29c_2_1"       1            3   
1                 "08944f89f5a69eb0f3d9_2_1"       1            2   
2     "0c2e896b3e86420d854609b41ffb7229_2_1"       1            1   
3     "0c2e896b3e86420d854609b41ffb7229_2_1"       2            1   
4     "0c2e896b3e86420d854609b41ffb7229_3_1"       2            1   
5    "0cb4a601e614492a8c2df8a7d024a012_10_1"       2            1   
6    "0cb4a601e614492a8c2df8a7d024a012_10_1"       1           22   
7    "0cb4a601e614492a8c2df8a7d024a012_10_1"       2           22   
8    "0cb4a601e614492a8c2df8a7d024a012_11_1"       2            1   
9    "0cb4a601e614492a8c2df8a7d024a012_11_1"       1           22   
10   "0cb4a601e614492a8c2df8a7d024a012_11_1"       2           22   
11   "0cb4a601e614492a8c2df8a7d024a012_12_1"       2            1   
12   "0cb4a601e614492a8c2df8a7d024a012_12_1"       1           22   
13   "0cb4a601e614492a8c2df8a7d024

In [16]:
percentage=percentage.to_json(orient='records')

In [17]:
print(percentage)

[{"question":"\"062bec6c9167fb11f29c_2_1\"","attempt":1,"numOfAnswers":3,"percentageOfRightAnswers":66.6666666667},{"question":"\"08944f89f5a69eb0f3d9_2_1\"","attempt":1,"numOfAnswers":2,"percentageOfRightAnswers":100.0},{"question":"\"0c2e896b3e86420d854609b41ffb7229_2_1\"","attempt":1,"numOfAnswers":1,"percentageOfRightAnswers":0.0},{"question":"\"0c2e896b3e86420d854609b41ffb7229_2_1\"","attempt":2,"numOfAnswers":1,"percentageOfRightAnswers":0.0},{"question":"\"0c2e896b3e86420d854609b41ffb7229_3_1\"","attempt":2,"numOfAnswers":1,"percentageOfRightAnswers":0.0},{"question":"\"0cb4a601e614492a8c2df8a7d024a012_10_1\"","attempt":2,"numOfAnswers":1,"percentageOfRightAnswers":100.0},{"question":"\"0cb4a601e614492a8c2df8a7d024a012_10_1\"","attempt":1,"numOfAnswers":22,"percentageOfRightAnswers":68.1818181818},{"question":"\"0cb4a601e614492a8c2df8a7d024a012_10_1\"","attempt":2,"numOfAnswers":22,"percentageOfRightAnswers":68.1818181818},{"question":"\"0cb4a601e614492a8c2df8a7d024a012_11_1\"",